<center><a href="https://rapids.ai/"><img src="img/rapids.png" alt="rapids_logo" style="width: 1500px;"/></a></center>

# <font color='#8735fb'> Data Acquisition

In this Notebook, we will be using a series of shell scripts to download the high-dimensional airline data. (U.S. Department of Transportation. "Airline Service Quality Performance 234 \[On-Time performance data\]", distributed by the U.S. Bureau of Transportation Statistics. 1987-2021. https://www.bts.dot.gov/browse-statistical-products-and-data/bts-publications/airline-service-quality-performance-234-time)

This dataset is open source and provided on an ongoing basis by the U.S. Bureau of Transportation Statistics.

Each month, the Bureau publishes a new `csv` file containing all flight information for the prior month. A single month of data may not be enough to build out a robust machine learning model. Our goal is to download several years' worth of this data and then combine it into a single `csv` file.

In addition to the flight data, we will also be downloading a file containing metadata and geo-coordinates of each airport and a file containing the code mappings for each airline. Airlines and airports rarely change, and as such, these files are static and do not change on a monthly basis. They do, however, contain information that we will later need to be mapped to the full airline dataset. (Megginson, David. "airports.csv", distributed by OurAirports. August 2, 2021. https://ourairports.com/data/airports.csv)

In [1]:
%%bash

# Get some basic information from the system we are running on
nvidia-smi
date

Fri Jul  2 22:34:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  Off  | 00000000:07:00.0 Off |                    0 |
| N/A   47C    P0    40W / 300W |      3MiB / 32505MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  Off  | 00000000:08:00.0 Off |                    0 |
| N/A   

In [2]:
# Set the location for data and name for final raw csv
data_dir = "/home/ec2-user/SageMaker/data"
csvfile_full = f"{data_dir}/airlines_raw_data_full.csv"

In [3]:
%%bash

# Files are stored on bts.gov with 1 file per-month, download each month for a subset of years
for month in `seq 1 8`; do # 12 for V100 32GB, 8 for T4 16GB
  for year in `seq 2019 2019`; do
    wget -q --no-check-certificate https://transtats.bts.gov/PREZIP/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_${year}_${month}.zip
    unzip -qou On_Time_Reporting_Carrier_On_Time_Performance_1987_present_${year}_${month}.zip
  done
done

Process is interrupted.


In [ ]:
%%bash -s "$data_dir"
data_dir="${1}"

# Remove the temporary .zip files and combine all CSV files into a single large file
rm -f On_Time_Reporting_Carrier_On_Time_Performance_1987_present_*zip*
mkdir -p "${data_dir}"
cat *csv > "${data_dir}/airlines_raw_data_full.csv"
rm On_Time_Reporting_Carrier_On_Time_Performance*csv

In [ ]:
%%bash -s "$data_dir"
data_dir="${1}"

# Get a single CSV file for the small-scale/POC training jobs
wget -q --no-check-certificate https://transtats.bts.gov/PREZIP/On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2019_6.zip
unzip -qou  On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2019_6.zip
mv "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2019_6.csv" "${data_dir}/airlines_raw_data.csv"

In [ ]:
%%bash -s "$data_dir"
data_dir="${1}"

# Download some static files that describe metadata about airports, airlines, and locations
cd "${data_dir}"
wget -q --no-check-certificate https://ourairports.com/data/airports.csv
wget -q --no-check-certificate  https://sagemaker-rapids-hpo-us-west-2.s3-us-west-2.amazonaws.com/airline_csv/carriers.csv # TODO: Get this from an official source

# <font color='#8735fb'> Data Exploration

Now that we have downloaded the data, we will do some basic exploration.

In the next few cells, we will take a look at how large the downloaded data is and what the headers or column names are.

Just get a feel for how big the data is and what type of information is included. It is unclear what some of the column names mean or how they might help our problem, but that is OK for now.

In [ ]:
%%bash -s "$data_dir"
data_dir="${1}"

# Print out some debug information about the downloaded files
ls -lh "${data_dir}/airlines_raw_data_full.csv"
ls -lh "${data_dir}/airlines_raw_data.csv"
ls -lh "${data_dir}/carriers.csv"
ls -lh "${data_dir}/airports.csv"
wc -l  "${data_dir}/airlines_raw_data_full.csv"
wc -l  "${data_dir}/airlines_raw_data.csv"
wc -l  "${data_dir}/carriers.csv"
wc -l  "${data_dir}/airports.csv"

In [ ]:
%%bash -s "$data_dir"
data_dir="${1}"

# Take a look at the headers for each of the CSV files
head -n 1 "${data_dir}/airlines_raw_data_full.csv"
echo
head -n 1 "${data_dir}/carriers.csv"
echo
head -n 1 "${data_dir}/airports.csv"

In [ ]:
%%bash -s "$data_dir"
data_dir="${1}"

# Look at the total file size of everything and determine how long the data download process took
du -sh "${data_dir}"
du -h "${data_dir}/"

date

# <font color='#8735fb'> Data Loading

Now that we have downloaded the flight data, compressed it all into a single ~4 GB file, and inspected the headers, let's verify that we can load this into a Python DataFrame.

First we will use the RAPIDS library. [cuDF](https://github.com/rapidsai/cudf) is a GPU DataFrame. This will use the NVIDIA GPU to load data directly into GPU memory.

We will then compare this to the pandas library. [pandas DataFrames](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) use the CPU to load data into system memory. This may be considerably slower, and as such, we do it last.

As a comparison, we time the two data-loading techniques. Loading the data into the GPU using RAPIDS should take around 10 seconds, while loading it into system memory with pandas may take up to 2 minutes.

In [ ]:
%%time

# Load data with Python and verify the contents
import cudf

data = cudf.read_csv(csvfile_full)

print(data.columns)
data.head()

In [ ]:
%%time
# Load data with Python and verify the contents
import pandas as pd

data = pd.read_csv(csvfile_full)

print(data.columns)
data.head()

# <font color='#8735fb'> Summary

We have now downloaded four data files and verified that they can be loaded into our program in later steps.

* `airlines_raw_data_full.csv`: The flight-data file we generated by combining each month of data
* `airlines_raw_data.csv`: A single-month subset of the flight data we will use later for faster iteration or prototyping
* `carriers.csv`: The file containing static carrier-specific data
* `airports.csv`: The file containing static airport-specific data

These will be used in the next module when we perform some pre-processing to combine these into a single, more-useful .csv file.

<center><a href="https://developer.nvidia.com/machine-learning"><img src="img/nvidia.png" alt="nvidia_logo" style="width: 1500px;"/></a></center>